In [12]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import uuid, pandas as pd

In [3]:
sntnc_trnsfrmr_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import uuid
from langchain_core.documents import Document

# Input data
qa_list = [
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "What is 2 + 2?", "answer": "4"},
    {"question": "Who wrote 'Harry Potter'?", "answer": "J.K. Rowling"},
    {"question": "What is the largest planet in our solar system?", "answer": "Jupiter"},
    {"question": "How many continents are there on Earth?", "answer": "7"}
]

# Create fake_whole_documents and docs
whole_docs = []
vector_store_docs = []

for qa in qa_list:
    unique_id = str(uuid.uuid4())
    whole_docs.append((unique_id, Document(page_content=str(qa))))
    vector_store_docs.append(Document(page_content=qa["question"], metadata={"doc_id": unique_id}))

docstore = InMemoryStore()
docstore.mset(whole_docs)    

In [ ]:
chrm_b = Chroma.from_documents(documents=vector_store_docs, embedding=sntnc_trnsfrmr_embeddings, collection_metadata={"hnsw:space": "cosine"})

In [5]:
# chroma_cache_vector = Chroma(persist_directory="./src/vector_dbs/chroma_cache_vector", embedding_function=sntnc_trnsfrmr_embeddings)
cache_path = "./chroma_cache_vector"
chroma_cache_vector = Chroma(persist_directory=cache_path, embedding_function=sntnc_trnsfrmr_embeddings, collection_metadata={"hnsw:space": "cosine"})

In [14]:
from langchain.storage import LocalFileStore

# Instantiate the LocalFileStore with the root path
cache_docstore = LocalFileStore(r"D:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\src\vector_dbs\cache_docs")

In [9]:
cache_docstore.mset([("key3", b"value1"), ("key4", b"value2")])

In [15]:
cache_docstore.mget(['367cf847-4803-483e-9361-f2362390613f'])

[b"content='The accuracy of CI1A J Thermocouple ranges from 0 to 115\xc2\xb0C (32 to 240\xc2\xb0F) with an accuracy of \xc2\xb12% or \xc2\xb13\xc2\xb0C (-6\xc2\xb0F).' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-01-14T18:41:00.8312193Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4708192200, 'load_duration': 3101218400, 'prompt_eval_count': 208, 'prompt_eval_duration': 188000000, 'eval_count': 41, 'eval_duration': 1416000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-3fb02d19-741f-426c-8769-7f2b781d2f51-0' usage_metadata={'input_tokens': 208, 'output_tokens': 41, 'total_tokens': 249}"]

In [7]:
def cache_update(question, answer):
    # Create fake_whole_documents and docs
    whole_docs = []
    vector_store_docs = []
    unique_id = str(uuid.uuid4())
    whole_docs.append((unique_id, Document(page_content=str(question))))
    vector_store_docs.append(Document(page_content=answer, metadata={"doc_id": unique_id}))

    cache_docstore.mset(whole_docs)  
    chroma_cache_vector.add_documents(vector_store_docs)

In [ ]:
def cache_retrival(question):
    rslt = chroma_cache_vector.similarity_search_with_score(question)
    rslt_df = pd.DataFrame(rslt, columns=["doc", "score"])  
    rlvnt_df = rslt_df[rslt_df["score"] <= 0.1]
    if rlvnt_df.empty:
        return None
    else:
        try:
            cache_id = rlvnt_df.head(1)['doc'].values[0].metadata['doc_id']
            answer = cache_docstore.get(cache_id).page_content
            return answer
        except:
            return None

In [16]:
from langchain_ollama import ChatOllama, OllamaLLM

In [17]:
llm_llama3 = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
    # other params...
)

In [31]:
import json

In [18]:
from langchain_core.messages import HumanMessage
ai_message = llm_llama3.invoke([HumanMessage("Tell me a joke")])
ai_message # <-- AIMessage

AIMessage(content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here."', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-01-14T18:49:02.963759Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2621389800, 'load_duration': 1549472000, 'prompt_eval_count': 29, 'prompt_eval_duration': 75000000, 'eval_count': 44, 'eval_duration': 994000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f3afc8af-89af-4cdd-bf21-7543d676afb4-0', usage_metadata={'input_tokens': 29, 'output_tokens': 44, 'total_tokens': 73})

In [19]:
ai_message.content

'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here."'

In [ ]:
import fitz  # PyMuPDF
from pptx import Presentation
from pptx.util import Inches

def pdf_to_ppt(pdf_file, ppt_file):
    pdf_document = fitz.open(pdf_file)
    presentation = Presentation()

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pix = page.get_pixmap()
        image_path = f"page_{page_num + 1}.png"
        pix.save(image_path)

        slide = presentation.slides.add_slide(presentation.slide_layouts[5])
        left = Inches(0.5)
        top = Inches(0.5)
        slide.shapes.add_picture(image_path, left, top, width=Inches(9), height=Inches(7))

    presentation.save(ppt_file)

pdf_to_ppt("input.pdf", "output.pptx")


In [ ]:
from PyPDF2 import PdfWriter, PdfReader

# Read the input PDF
input_pdf = PdfReader("document.pdf")

# Iterate through each page and create a new PDF file for each page
for i in range(len(input_pdf.pages)):
    output = PdfWriter()
    output.add_page(input_pdf.pages[i])
    
    # Write the new PDF file
    with open(f"document-page{i}.pdf", "wb") as output_stream:
        output.write(output_stream)
